<a href="https://colab.research.google.com/github/raghav-n/singapore-avi-ebird/blob/master/scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
### DOWNLOADS/IMPORTS

# YEARS COMPLETED: 2005, 1998

! apt-get install poppler-utils
! sudo apt install tesseract-ocr
! sudo apt-get install tesseract-ocr
! pip install pdf2image
! pip install pytesseract
! wget "https://nss.org.sg/report/7f6f74f5-7Avi_2000_14_1.pdf" # DONE
# ! wget "https://nss.org.sg/report/68d7eabd-0Avi_1997_11_1.pdf" # DONE
# ! wget "https://nss.org.sg/report/e72a1102-aAvi_1997_11_2.pdf" # DONE
# ! wget "https://nss.org.sg/report/7b56a329-8Avi_1998_11_3.pdf" # DONE
# ! wget "https://nss.org.sg/report/29644f6a-bAvi_1998_11_4.pdf" # DONE
! pip install word2number
% load_ext google.colab.data_table

import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('maxent_treebank_pos_tagger') 

from google.colab import drive
import pandas as pd
import pickle
import numpy as np

drive.mount('/content/gdrive')
df_in = pd.read_csv("/content/gdrive/My Drive/df (10).csv")
datafile = pd.read_csv("/content/gdrive/My Drive/ebird-checklist.csv", encoding= 'unicode_escape')
cachedsci = np.load('/content/gdrive/My Drive/cachedsci.npy',allow_pickle='TRUE').item()
cachedeng = np.load('/content/gdrive/My Drive/cachedeng.npy',allow_pickle='TRUE').item()
cachedpicks = np.load('/content/gdrive/My Drive/cachedpicks.npy',allow_pickle='TRUE').item()
contribs = np.load('/content/gdrive/My Drive/contributors.npy',allow_pickle='TRUE').item()
hotspots = pd.read_csv("/content/gdrive/My Drive/lookup-hotspots.csv", encoding= 'unicode_escape')
% cd ..

from pdf2image import convert_from_path, convert_from_bytes
import cv2
import pytesseract
import numpy as np
import re
from google.colab.patches import cv2_imshow
from tqdm.notebook import tqdm
from functools import partial
import numpy as np

import pprint
from nltk.tokenize import word_tokenize
from nltk import pos_tag, ne_chunk
import pandas as pd
import nltk.tag, nltk.data
import copy
from word2number import w2n

from google.colab import files
from datetime import datetime
import editdistance
from tqdm.notebook import tqdm
from IPython.display import display, HTML, clear_output
import time
import pickle
from google.colab import data_table

In [ ]:
### OCR

tqdm = partial(tqdm, position=0, leave=True)

try:
  images = convert_from_bytes(open('7f6f74f5-7Avi_2000_14_1.pdf', 'rb').read())
except:
  images = convert_from_bytes(open('content/7f6f74f5-7Avi_2000_14_1.pdf', 'rb').read())

text = ''

nu = 0

contains = re.compile(r'((?:[A-Z]?[a-z]+ ?){1,5}\(([A-Za-z ]+)\))')

for i in tqdm(images[2:24]): # include the contributors page!
  open_cv_image = np.array(i) 
  img = open_cv_image[:, :, ::-1].copy()
  img = cv2.resize(img, None, fx=1.5, fy=1.5, interpolation=cv2.INTER_CUBIC)

  gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
  # gray = cv2.medianBlur(gray, 3)
  gray, img_bin = cv2.threshold(gray,128,255,cv2.THRESH_BINARY | cv2.THRESH_OTSU)

  gray = cv2.bitwise_not(img_bin)

  kernel = np.ones((2, 1), np.uint8)
  img = cv2.erode(gray, kernel, iterations=1)
  img = cv2.dilate(img, kernel, iterations=1)
  tt = pytesseract.image_to_string(cv2.bitwise_not(img))
  if nu != 0:
    text += (tt.replace("/\n","/") + "\n\n")
  else: 
    text += (tt.replace("/\n","/") + "\n\n")
    contributors = (tt.replace("\n"," "))
    if (contains.findall(contributors)) == []:
      print("Couldn't find any contributors on the first page...")
      raise ValueError
  nu += 1

contribs2 = contains.findall(contributors)
contribs2 = [[b.replace('(', '').replace(')', ''), a.replace(f'({b})', '')] for (a, b) in contribs2]
contribs2[-1][1] = contribs2[-1][1].replace(' and ', '')
contribs2 = dict(contribs2)

for k in contribs2.keys():
  contribs[k] = contribs2[k]
  if 'i' in k:
    contribs[k.replace('i', 'l')] = contribs[k]
  if 'l' in k:
    contribs[k.replace('l', 'i')] = contribs[k]

for k in contribs.keys():
  text = text.replace(k, k.upper())
  text = re.sub(re.compile("[(/]"+k+"[/)]", flags=re.IGNORECASE), lambda x : x.group(0).upper(), text)

np.save("content/contributors.npy", contribs)

print("Review contributors, check for errors:")
contribs

In [ ]:
# text = text.replace("PFR", "Pasoh Forest Reserve")
# newtext = ''
# for line in text.splitlines():
#   if ("Pasoh Forest Reserve" in line or "GTB" in line) and "(" not in line and "(LKC/SR)" not in line:
#     newtext += line.replace(".", ". (LKC/SR)")
#   else:
#     newtext += line
#   newtext += "\n"
# text = newtext

In [11]:
### DATA PT. 1
import textwrap

res2005 = re.compile(r'^([A-Z -]){5,}.*?[A-Z]?[a-z ]*$')
res2000 = re.compile(r"^[A-Z -]+([A-Z]?[a-z]+[ '-])*[/a-z]+$")
res2 = re.compile(r"^\S+[- ]+\S+[- ]+\S+[- ]+\S+[- ]*\S*$")
splitter = re.compile(r'([({]a?l?l?[A-Z/ ]*[)/}]|\(sev obs\)|\(both by [A-Zdl/]+\)|\(both [A-Zdl/]+\))')
date = re.compile(r'([0-9]{1,2} ?/ ?[0-9]{1,2})|(Multidate\[.*\])')
address_re = re.compile(r'( [A-Z][a-z]+)+ ((Road)|(Lane)|(Street)|(Avenue)) [0-9]+')
volume = re.compile(r'Volume \d+ No \d+')
rang = re.compile(r'([0-9]{1,2} ?/ ?[0-9]{1,2}) to ([0-9]{1,2} ?/ ?[0-9]{1,2})')
rang2 = re.compile(r'([0-9]{1,2})-([0-9]{1,2} ?/ ?([0-9]{1,2}))')
multi = re.compile(r'((([0-9]{1,2} ?/ ?[0-9]{1,2}),? ?a?n?d? ?){2,})')
multisub = re.compile(r'Multidate\[(.*)\]')

USERES = res2000
BACKUPRES = res2

def flatten(l):
  return [item for sublist in l for item in sublist]

def address(string, address_re):
  try:
    addr = address_re.search(string).group()
    return ([addr], [j for j in addr.split(' ') if j.isdigit()])
  except AttributeError:
    return [[], []]

class Sighting:
  def __init__(self, species, date, count, person, location, string=None):
    self.person = person
    self.date = date
    self.species = species
    self.count = count
    self.string = string
    self.location = location
    self.data = [species, date, count, person, location, string]
  def __str__(self):
    return f"SPECIES: {self.species}\nDATE: {self.date}\nCOUNT: {self.count}\nPERSON: {self.person}\nLOCATION: {self.location}\nCOMMENTS: {self.string}"
  def join(self, other):
    data2 = zip(self.data, other.data)
    datanew = []
    n0 = 0
    for a, b in data2:
      if n0 == 3:
        n0 += 1
        if a in ['', 0] and b not in ['', 0]:
          datanew.append([b, b])
        else:
          datanew.append([a, b])
        continue
      if a in ['', 0] and b not in ['', 0]:
        datanew.append([b, b])
      elif a not in ['', 0] and b in ['', 0]:
        datanew.append([a, a])
      else:
        datanew.append([a, b])
      n0 += 1
    return (Sighting(*[a for a, b in datanew]), Sighting(*[b for a, b in datanew]))

def make_df(sightings):
  d = {
      "Species": [i.species for i in sightings],
      "Date": [i.date for i in sightings],
      "Count": [i.count for i in sightings],
      "Observer": [i.person for i in sightings],
      "Location": [i.location for i in sightings],
      #"Comments": [i.string for i in sightings]
  }
  df = pd.DataFrame.from_dict(d)
  return df

def get_multidates(tex):
  out = flatten(rang.findall(tex))
  out2 = flatten(rang2.findall(tex))
  out3 = [i for i in flatten(multi.findall(tex)) if i != 'and' and i != '']
  if out !=  []:
    return rang.sub(r"Multidate[\1,\2]", tex)
  if out2 != []:
    return rang2.sub(r"Multidate[\1/\3,\2]", tex)
  if out3 != []:
    txt = multi.sub(r"Multidate[\1] ", tex)
    for it in multisub.findall(txt):
      txt = re.sub(',+', ',', txt.replace(it, it.replace(" ", ",").replace('and', ''))).replace(',]', ']')
    return txt
  return tex

def look(arr, item, prev = 0):
  for i in range(prev+1, len(arr)):
    if item in arr[i]:
      return i - 1
  print(arr, item)

spdata = {}
spdata2 = []

latest = ''
text = re.sub("Page [0-9]+ SINGAPORE AVIFAUNA", "", text)
text = re.sub("SINGAPORE AVIFAUNA Page [0-9]+", "", text)
text = text.replace("SINGAPORE AVIFAUNA", '')

if spdata == {}:
  for t in text.split("\n"):
    if re.match(USERES, t.strip())!=None or re.match(USERES, t.strip()[:-5])!=None or re.match(USERES, t.strip().replace('’', '\''))!=None:
      if t not in spdata.keys():
        spdata[t] = ''
      latest = t
    elif re.match(BACKUPRES, t.strip())!=None and "Page" not in t:
      ask = input("Is this a species? " + t.strip() + "\n").strip()
      if ask.lower() != '':
        if t not in spdata.keys():
          spdata[t] = ''
        latest = t
      elif latest != '':
        spdata[latest] += (' ' + t)
    elif latest != '':
      spdata[latest] += (' ' + t)

n = 0

for k, iv in list(spdata.items()):
  va = False
  n += 1
  #if len(date.findall(i)) <= len(splitter.findall(i)):
  iv = iv.replace('[','1').replace(']', '1').replace("|", '1')
  s = splitter.split(iv)
  i = get_multidates(iv)
  dates = [k for k in flatten(date.findall(i)) if k != '']
  
  while len(s[1::2]) != len(dates):
    print("Revise text")
  
    for wrap in textwrap.wrap(iv):
      print(wrap)
    inp2 = input().strip()
    if inp2 == '':
      va = True
      break
    
    iv = inp2
    s = splitter.split(iv)
    i = get_multidates(iv)
    dates = [k for k in flatten(date.findall(i)) if k != '']
  
  if va:
    continue

  li = list(zip(s[::2], s[1::2], dates))
  li = [(a.replace(c,''), b, c) for a, b, c in li ]

  # s2 = date.split(i)
  # dates2 = (date.findall(i))
  # obs2 = splitter.findall(i)
  # li2 = list(zip(s2, obs2, dates2))
  # li2 = [(a.replace(c,''), b,  c) for a, b, c in li2 ]
  
  # prev = 0
  # numb = 0

  # for i in obs2:
  #   index = look(obs2, i, prev)
  #   li2[index].append(obs2[numb])
  #   numb += 1

  # li2 = [[s2_val, obs2_val, dates2_val] for (s2_val, dates2_val, ovs2_val) in li2]

  # if len(obs2) < len(dates):
  #   print(s2, dates2, obs2)
  #   li = copy.deepcopy(li2)

  for string1, person, d in li:
    inp1=''
    string = volume.sub('', string1)
    matched, ignore = address(string, address_re)
    matched = [i1.strip() for i1 in matched]
    if "Track 270" in string:
      matched.append("Track 270")
      ignore.append("270")
    if "Track 246" in string:
      matched.append("Track 246")
      ignore.append("246")
    original = copy.deepcopy(string)
    string = string.rstrip(', ').lstrip(', ')
    for repl in ["a pair", "1 pair", "another pair", "pair", "1 male and 1 female", "a male and a female", "an adult and a juvenile", "1 seen and 1 heard"]:
      string = re.compile(repl, re.IGNORECASE).sub('2', string)
    for repl in ["several", "a number"]:
      string = re.compile(repl, re.IGNORECASE).sub('1111', string)
    string = string.replace("Singapore Avifauna", '')
    string = string.replace("Report", '')
    for repl in ["a juvenile", "a subadult", "a sub-adult", "a female", "a male", "a summer male", "another", "\|", "an immature", "an adult", "singles", " \["]:
      string = re.compile(repl, re.IGNORECASE).sub(' 1', string)
    string = re.compile("including \d", re.IGNORECASE).sub('', string)
    if all([word in ["and", ",", ''] for word in string.split(" ")]) or len(string) == 0:
      spdata2.append(Sighting(k, d, 0, person, '', string=string))
      continue
    location = matched
    count = []
    wtk = word_tokenize(string)
    pos_tag1 = pos_tag(wtk)
    chunk = ne_chunk(pos_tag1)
    for ix in chunk.subtrees(filter=lambda x: x.label() in ["PERSON", "ORGANIZATION", "FACILITY", "LOCATION"]):
      location.append(" ".join([a for a, b in ix]))
    if len(location) == 0:
      for ix in wtk:
        if ix[0].isupper():
          location.append(ix)
      location = [" ".join(location)]
    location = [l for l in location if l not in ["Sungei Puaka", "Kg Melayu", "Nesting Report", "Ficus", "Trail", "African Tulip Tree"]]
    for ix in chunk:
      if len(ix) > 1 and (ix[1] == "CD") and '/' not in ix[0]:
        count.append(ix[0])
    for num in ignore:
      count.remove(num)
    string = re.compile("[0-9]+ ?meters", re.IGNORECASE).sub('', string)
    string = re.compile("[0-9]+ ?metres", re.IGNORECASE).sub('', string)
    if len(count) != 1:
      while True:
        inp = input(f"COUNT for {string.strip()}      {n}/{len(spdata.items())}\n").strip()
        if inp == 'skip': 
          break
        try: 
          if inp == '':
            count = 0
            break
          count1 = int(inp)
          break
        except:
          if inp == 'more':
            print(k, i)
          continue
    else:
      inp = count[0].replace('.', '')
    if count != 0 and inp != 'skip': 
      try:
        count = int(inp)
      except ValueError:
        try:
          count = w2n.word_to_num(inp)
        except ValueError:
          while True:
            inp = input(f"COUNT for {string.strip()}      {n}/{len(spdata.items())}\n").strip()
            if inp == 'skip': 
              break
            try: 
              if inp == '':
                count = 0
                break
              count1 = int(inp)
              break
            except:
              if inp == 'more':
                print(k, i)
              continue
    for place in location:
      if len(matched) > 0 and place in matched[0] and place != matched[0] and place != '':
        location.remove(place) # remove subsets

    location = list(set(location))
    location = [loc for loc in location if loc != 'Multidate']
    if len(location) > 1:
      try:
        l1 = copy.deepcopy(location)
        location = cachedpicks[tuple(location)]
      except KeyError:
        while True:
          inp = input(f"LOCATION for {string.strip()}; options: {location}      {n}/{len(spdata.items())}\n").strip()
          if inp == 'skip': 
            break
          if inp == 'other':
            inp1 = 'other'
            inp = input(f"NEW LOCATION for {string.strip()}; options: {location}      {n}/{len(spdata.items())}\n").strip()
            location = inp
            break
          try: 
            if inp == '':
              location = ''
              break
            location1 = location[int(inp)]
            break
          except:
            if inp == 'more':
              print(k, i)
            continue
    else:
      if inp != 'skip':
        inp = 0
    try:
      if location != '' and inp != 'skip' and type(location) == list:
        cachedpicks[tuple(location)] = copy.deepcopy(location[int(inp)])
        location = location[int(inp)]
    except ValueError:
      location = " ".join([ location[i] for i in map(int, inp.split(',')) ])
    if inp != 'skip':
      if inp1 == 'other':
        print(f"Switched location to {location}")
      if type(location) == list:
        location = location[0]
      sight = Sighting(k, d, count, copy.deepcopy(person), copy.deepcopy(location), string=string)
      spdata2.append(sight)
    else:
      print("skipped this one")

for x in range(1, len(spdata2)):
  spdata2[x], spdata2[x-1] = spdata2[x].join(spdata2[x-1])

np.save("content/cachedpicks.npy", cachedpicks)

Is this a species? BTNR - Bukit Timah Nature Reserve

Is this a species? BBNP - Bukit Batok Nature Park

Is this a species? Tanah Merah, 30/1 (AC/AOY/KK/YPL).

Is this a species? Black-winged Kite E/anus caeruleus
1
Is this a species? sightings were made in bird-waves.

Is this a species? 1 at BBNP, 8/2 (LKC).

Is this a species? 4 at Poyan, 6/2 (LKS/SBS/TT).

Is this a species? "Several" at Poyan, 6/2 (LKS/SBS/TT).

Is this a species? 1 at BBNP, 25/2 (CA).

Is this a species? seen since last Christmas.

Is this a species? TT, tetttereteeee ee A

Is this a species? "eyelash" present in Scarlet-rumped.

Is this a species? Blue-banded Kingfisher A/cedo euryzona
1
Is this a species? . Malaysian Eared-nightjar Eurostopodus temmincki
1
Is this a species? 1 at BTNR, 2/3 (CA).

Is this a species? 1 at BBNP, 9/3 (CA).

Is this a species? some 20km north of here.

Is this a species? Compiled by Lim Kim Chuah.

Revise text
 2000  This report covers noteworthy bird sightings in the first
quarter 

In [15]:
### DATA PT. 2
from tqdm.notebook import tqdm

tqdm.pandas()

# cachedeng = {k: list(datafile.loc[datafile["scientific name"] == v]["English name"])[0] for k, v in cachedsci.items()}
# print(cachedeng)

# a_file = open("cachedeng.txt", "wb")
# pickle.dump(cachedeng, a_file)
# a_file.close()

overrides = {
    "Lonchura molucca":         "Lonchura atricapilla",
    "Black-faced Munia":        "Black-headed Munia",
    "Nilgiri Flowerpecker":     "Plain Flowerpecker",
    "Dicaeum concolor":         "Dicaeum minullum",
    "Collared Scops-Owl":       "Sunda Scops-Owl",
    "Otus lettia":              "Otus lempiji",
    "Glossy Swiftlet":          "Plume-toed Swiftlet",
    "Collocalia esculenta":     "Collocalia affinis",
    "Indian Paradise-Flycatcher": "Amur/Blyth's Paradise-Flycatcher",
    "Terpsiphone paradisi":     "Terpsiphone incei/affinis",
    "Ashy Bulbul":              "Cinereous Bulbul",
    "Hemixos flavala":          "Hemixos cinereus",
    "Jungle Nightjar":          "Gray Nightjar",
    "Caprimulgus indicus":      "Caprimulgus jotaka",
    "Narcissus Flycatcher":     "Green-backed Flycatcher",
    "Ficedula narcissina":      "Ficedula elisae",
    "Blue-and-white Flycatcher": "Blue-and-white/Zappey's Flycatcher",
    "Cyanoptila cyanomelana":   "Cyanoptila cyanomelana/cumatilis",
    "Siberian Stonechat":       "Siberian Stonechat (Stejneger's)",
    "Saxicola maurus":          "Saxicola maurus stejnegeri",
    "Brown Barbet":             "Sooty Barbet",
    "Caloramphus fuliginosus":  "Caloramphus hayii"
}

cachedsci = {k: v if v not in overrides.keys() else overrides[v] for (k, v) in cachedsci.items()}
cachedeng = {k: v if v not in overrides.keys() else overrides[v] for (k, v) in cachedeng.items()}
flipped_c = {v: k for k, v in contribs.items()}

done = 0

def get(spname):
  global done
  datafile2 = datafile.copy(deep=True)
  text = spname.lower()
  datafile2["Distance"] = datafile2["Pair"].apply(lambda i: editdistance.eval(text, i))
  datafile2 = datafile2.sort_values(by='Distance')
  vsci = list(datafile2["scientific name"])[0]
  veng = list(datafile2["English name"])[0]
  score = list(datafile2["Distance"])[0]
  if score > 5:
    try:
      print(text, done)
      n = 3
      display(datafile2.head(3))
      inpu = input("Confirm?").strip()
      while inpu == 'more':
        n += 5
        display(datafile2.head(n))
        inpu = input("Confirm?").strip()
      if inpu != '':
        vsci = list(datafile2["scientific name"])[int(inpu)]
        veng = list(datafile2["English name"])[int(inpu)]
        inpu2 = input("Confirm again").strip()
        if inpu != '':
          vsci = list(datafile2["scientific name"])[int(inpu2)]
          veng = list(datafile2["English name"])[int(inpu2)]
    except ValueError:
      text2 = input("Species name").strip()
      datafile2["Distance"] = datafile2["Pair"].apply(lambda i: editdistance.eval(text2, i))
      datafile2 = datafile2.sort_values(by='Distance')
      vsci = list(datafile2["scientific name"])[0]
      veng = list(datafile2["English name"])[0]
      score = list(datafile2["Distance"])[0]
      if score > 5:
          print(text2)
          n = 3
          display(datafile2.head(3))
          inpu = input("Confirm?").strip()
          while inpu == 'more':
            n += 5
            display(datafile2.head(n))
            inpu = input("Confirm?").strip()
          if inpu != '':
            vsci = list(datafile2["scientific name"])[int(inpu)]
            veng = list(datafile2["English name"])[int(inpu)]
            inpu2 = input("Confirm again").strip()
            if inpu != '':
              vsci = list(datafile2["scientific name"])[int(inpu2)]
              veng = list(datafile2["English name"])[int(inpu2)]
        
  cachedsci[text] = vsci
  cachedeng[text] = veng
  clear_output()
  return (vsci, veng)

contributors1 = {k.upper(): v for k, v in contribs.items()}

def expand(obser):
  if type(obser) == str:
    obser = obser[1:-1]
  else:
    return None
  text = []
  for o in obser.split('/'):
    try:
      text.append(contributors1[o.replace(' ', '')].strip())
    except:
      text.append(o)
  return '/'.join(text)

def optimum(spname, scieng):
  global done
  done += 1
  global cachedsci, cachedeng
  try:
    if scieng == 'sci':
      return cachedsci[spname.lower()]
    if scieng == 'eng':
      return cachedeng[spname.lower()]
  except KeyError:
    if scieng == 'sci': 
      return get(spname)[0]
    if scieng == 'eng': 
      return get(spname)[1]

made = make_df([i for i in spdata2 if type(i) == Sighting])
made["Comments"] = made["Species"].apply(lambda i: spdata[i].strip())
made["Observer(s)"] = made["Observer"].apply(expand)

made['Date'] = made['Date'].str.replace("Multidate\[", '').str.replace("]",'').str.split(',')

# made = (made
#             .set_index([i for i in made.columns if i != "Date"])['Date']
#             .apply(pd.Series)
#             .stack()
#             .reset_index()
#             .drop('level_6', axis=1)
#             .rename(columns={0:'Date'}))

# 

try:
  made = made.explode('Date')[df_in.columns]
except:
  made = made.explode('Date')

join1 = True

if join1:
  df = pd.concat([df_in, made])
else:
  df = made.copy(deep=True)

YEAR = 0

def dt(row):
  global YEAR
  d = row["Date"]
  if type(d) == str and "Multidate" in d:
    return d
  if type(d) == str:
    try:
      if d.count('/') == 1:
        if YEAR == 0:
          YEAR = int(input("What year were the added sightings seen in?\n").strip())
        return datetime(year=YEAR,month=int(d.split("/")[1]),day=int(d.split("/")[0]))
      elif d.count('/') == 2:
        return datetime(year=int(d.split("/")[2]),month=int(d.split("/")[0]),day=int(d.split("/")[1]))
      if d.count('-') == 2:
        return datetime(year=int(d.split("-")[0]),month=int(d.split("-")[1]),day=int(d.split("-")[2]))
    except:
      return None
  else:
    return d

df["Date"] = df.apply(dt,axis=1)

datafile["Pair"] = datafile["Pair"].apply(lambda i: i.lower())
df["English name"] = df["Species"].progress_apply(lambda i: optimum(i, 'eng'))
df["Scientific name"] = df["Species"].apply(lambda i: optimum(i, 'sci'))
df = df[df["English name"] != "Brubru"]
df["Count"] = df["Count"].apply(lambda i: "X" if i == 1111 or i == 111 else i)
df["Observer(s)"] = df["Observer(s)"].apply(lambda i: "Several observers" if i == 'sev obs' else i)
df["Observer(s)"] = df["Observer(s)"].apply(lambda i: expand('('+i.replace("all ", '')+')') if i.replace("all ", '').upper() == i else i)

df = df[~(df["Date"]==None)]

np.save("content/cachedsci.npy", cachedsci)

np.save("content/cachedeng.npy", cachedeng)

df = df[[co for co in df.columns if co != "Observers2"]]

df["Count"] = df["Count"].apply(lambda i: i if type(i) == int or str(i).isdigit() else 'X')

cols = ["English name", "Scientific name", "Date", "Count", "Observer(s)", "Location", "Comments"]
cols2 = ["Species", "Date", "Count", "Observer(s)", "Location", "Comments"]

df_display = df[cols]

df_display = df_display[~df_display["Date"].isnull()]

def process(item):
  d = item.Date
  try:
    date = d.strftime("%d").lstrip("0") 
  except:
    print(item)
    raise

  month = d.strftime("%m").lstrip("0")
  dynamic_regex = re.compile(date + ' ?/ ?' + month)
  a = []

  for i in item.Comments.replace('St.', 'St').split('.'):
    if dynamic_regex.search(i):
      a.append([i.strip(), item.Location in i])

  if len(a) > 1:
    a = [[i, j] for (i, j) in a if j]

  a = " /// ".join([c for c, b in a])
  
  # if a == '':
  #   print(item["English name"] + " was removed.")
  #   # print("More details: ")
  #   # print(item.Location)
  #   # print(item.Comments)
  #   # print(str(item.Date).split(' ')[0] + "\n")
  #   a = "None"
  if a != '':
    item[-1] = a
  return item

def trip(i):
  vol = i.year - 1986
  return f'These sightings are reproduced with permission from Volume {vol} of Singapore Avifauna, which was published originally by Nature Society (Singapore).\
            The original report is available at https://nss.org.sg/report.aspx.'

df_display = df_display.apply(process, axis=1)
df_display = df_display[df_display["Comments"] != "None"]
df_display["Trip comments"] = df_display["Date"].apply(trip) 

df_display = df_display.drop_duplicates()
df = df.drop_duplicates(subset=["Date", "Species", "Location"])

df_in = df.copy(deep=True)

df[cols2].to_csv("content/df (10).csv", index=False)
# files.download("content/df (10).csv")

df_display.to_csv("content/output.csv", index=False)
# files.download("content/output.csv")

df1998 = df_display[df_display["Date"].dt.year.isin([1997, 1998, 2000])]
df1998.to_csv("content/output-spec.csv", index=False)

data_table.DataTable(df1998, include_index=False)

What year were the added sightings seen in?
2000


,English name,Scientific name,Date,Count,Observer(s),Location,Comments,Trip comments
0,Banded Bay Cuckoo,Cacomantis sonneratii,1998-01-04,1,Lim Kim Seng,Bukit Timah,"Singles seen at the summit of Bukit Timah, 4/1...",These sightings are reproduced with permission...
1,Banded Bay Cuckoo,Cacomantis sonneratii,1998-01-11,1,SBG,Sime Road,"Singles seen at the summit of Bukit Timah, 4/1...",These sightings are reproduced with permission...
2,Banded Bay Cuckoo,Cacomantis sonneratii,1998-01-25,1,Alfred Chia,Sime Road,Singles were also heard at Sime Road on 25/1 (...,These sightings are reproduced with permission...
3,Banded Bay Cuckoo,Cacomantis sonneratii,1998-01-29,1,Lim Kim Seng/Trixie Tan,Poyan,Singles were also heard at Sime Road on 25/1 (...,These sightings are reproduced with permission...
4,Banded Bay Cuckoo,Cacomantis sonneratii,1998-02-15,2,SBG,Lorong Sesuai,"2 seen low in trees at Lorong Sesuai, near Buk...",These sightings are reproduced with permission...
...,...,...,...,...,...,...,...,...
453,Yellow-vented Flowerpecker,Dicaeum chrysorrheum,2000-03-16,1,Chris Artuoso,Panti Forest,"1 feeding on a low bush, Panti Forest (Bunker ...",These sightings are reproduced with permission...
454,Streaky-breasted Spiderhunter,Arachnothera affinis,2000-03-14,1,Lim Kim Chuah,Panti Forest,"1 at Panti Forest (Bunker Track), 14/3 (LKC)",These sightings are reproduced with permission...
159,Straw-headed Bulbul,Pycnonotus zeylanicus,2000-02-05,2,Lim Kim Seng/Safizah Abdullah,BBNP,"At BBNP, 5 birds were counted on 5/2 (LKS/SA) ...",These sightings are reproduced with permission...
352,Crested Fireback,Lophura ignita,2000-02-23,X,Lim Kim Chuah,Tahan River,"1 male foraging along Bukit Teresik Trail, 21/...",These sightings are reproduced with permission...


In [18]:
### CORRECTIONS/PROOFREADING

### 1. CORRECT HOTSPOTS/SPECIES:

import copy

def add_info(row,df,enter=False,do_all=True,add='',add_to_lookup=False,country=0,orig=''):
  global hotspot_lookup
  row2 = row.copy(deep=True)
  if row2["Count"] != 'X' and (int(row2["Count"]) > 1000 or row2["Count"] in ["246", 246] or row2["Count"] in ["11", 11]):
    print("Comments: ", row2["Comments"], "Date: ", row2["Date"], "Count: ", row2["Count"])
    try:
      newc = input("Updated count?\n").strip()
    except:
      newc = input("Updated count?\n").strip()
    row2["Count"] = newc
  if row2["English name"] == "Hodgson's Hawk-Cuckoo":
    print(row2["Comments"])
    inp = ''
    while inp not in ["H", "M", "N"]:
      inp = input("Hodgson's (H), Malaysian (M), or Neither (N)?\n")
    if inp == "H":
      pass
    if inp == "M":
      row2["English name"] = "Malaysian Hawk-Cuckoo"
      row2["Scientific name"] = "Hierococcyx fugax"
    if inp == "N":
      row2["English name"] = "hawk-cuckoo sp."
      row2["Scientific name"] = "Hierococcyx sp."
      row2["Comments"] += " (Hodgson's/Malaysian Hawk-Cuckoo)"
  if row2["Location"] in list(hotspot_lookup["Location"]) or enter:
    lo = row2["Location"]
    if enter and row2["Location"] not in list(hotspot_lookup["Location"]):
      lo = row2["Location"]
      upd = lo
      more = add
      if add_to_lookup and orig != '':
        hotspot_lookup.loc[len(hotspot_lookup)] = [orig, more, upd, country]
      row2["Country"] = country
    else:
      looku = hotspot_lookup.loc[hotspot_lookup["Location"]==lo]
      upd = looku["Hotspot"].values[0]
      more = looku["what to add"].values[0]
      row2["Country"] = looku["Country"].values[0]
    more = ("" if pd.isna(more) else more)
    row2["Updated location"] = (upd if upd != 'retain' else row2["Location"])
    row2["Additional location info"] = more
    text2 = f'<b>Additional location info: </b>{more}' if more != '' else ''
    format = f'<b>Comments: </b>{row2["Comments"]}<br><b>Observer(s): </b>{row2["Observer(s)"]}<br>{text2}' 
    row2["Updated comments"] = format
  else:
    if do_all:
      index = (df[df["Comments"]==row2["Comments"]].index.values.astype(int)[0])
      print("Previous:", df.iloc[index-1]["English name"], df.iloc[index-1]["Comments"], "Location: ", df.iloc[index-1]["Location"])
      print("Current:", row2["English name"], row2["Comments"], "Location: ", row2["Location"])
      orig = copy.deepcopy(row2["Location"])
      lo = input("Location info?\n").strip()
      add_loc = input("Any additional location info?\n").strip()
      add_lookup = input("Add to the lookup?\n").strip() != ''
      country = input("What country?\n").strip()
      row2["Location"] = lo
      return add_info(row2, df, enter=True,add=add_loc,add_to_lookup=add_lookup,orig=orig,country=country)
    else:
      row2["Updated location"] = "?"
      row2["Additional location info"] = "?"
      row2["Country"] = "?"
      hotspot_lookup.loc[len(hotspot_lookup)] = [row2["Location"], "", "?", "?"]
  return row2

first = {
    "Bukit Kalang Service": "Bukit Kalang Service Reservoir"
}

df2005 = df1998.copy(deep=True)

df2005 = df2005.reset_index(drop=True)
df2005["Location"] = df2005["Location"].apply(lambda loc: first[loc] if loc in first.keys() else loc)

hotspot_lookup = pd.read_csv("/content/gdrive/My Drive/lookup-hotspots.csv")

hotspot_lookup = hotspot_lookup[[col for col in hotspot_lookup.columns if "Unnamed" not in col and col != "Context"]]
data_table.DataTable(df2005, include_index=False)

df2005["Country"] = df2005["Count"]

df2005 = df2005.apply(lambda row: add_info(row, df2005, do_all=False),axis=1)
hotspot_lookup.to_csv("content/lookup-hotspots.csv")

empty = ["" for _ in df2005["English name"]]

country = {
    "Singapore": "SG",
    "Indonesia": "IN",
    "Malaysia": "MY",
    "?": "?"
}

ebird_upload = pd.DataFrame(columns=["A: Common Name"])
ebird_upload["A: Common Name"] = df2005["English name"]
ebird_upload["B: Genus"] = empty
ebird_upload["C: Species"] = df2005["Scientific name"]
ebird_upload["D: Species Count"] = df2005["Count"]
ebird_upload["E: Species Comments"] = df2005["Updated comments"]
ebird_upload["E1: Comments"] = df2005["Comments"].str.replace("<b>","").str.replace("<br>","").str.replace("</b>","")
ebird_upload["F: Location Name"] = df2005["Updated location"]
ebird_upload["F1: Original Location (removed for upload)"] = df2005["Location"]
ebird_upload["G: Latitude"] = empty
ebird_upload["H: Longitude"] = empty
ebird_upload["I: Observation Date"] = df2005["Date"].dt.strftime("")
ebird_upload["J: Start Time"] = empty
ebird_upload["K: State/Province"] = ["" for i in df2005["Country"]]  # ASSUMING ONLY SG/JOHOR
ebird_upload["L: Country"] = [country[i] if i in country.keys() else "SG" for i in df2005["Country"]] # ASSUMING ONLY SG/MALAYSIA
ebird_upload["M: Protocol"] = ["historical" for i in df2005["Country"]]
ebird_upload["N: Number of Observers"] = [len(i.split('/')) if i != "sev obs" and i != "SBG" else "" for i in df2005["Observer(s)"] ]
ebird_upload["O: Duration"] = empty
ebird_upload["P: All Observations Reported?"] = ['N' for i in df2005["Country"] ]
ebird_upload["Q: Distance Covered"] = empty
ebird_upload["R: Area Covered"] = empty
ebird_upload["S: Checklist Comments"] = df2005["Trip comments"]

df2005["Updated location"] = ebird_upload["F: Location Name"]
df2005 = df2005[['English name', 'Scientific name', 'Date', 'Count', 'Observer(s)',
       'Location', 'Updated location', 'Comments', 'Trip comments']]
df2005.to_csv("content/output-spec.csv", index=False)

ebird_upload.to_csv("content/upload-spec.csv", index=False)
data_table.DataTable(ebird_upload, include_index=False)

Comments:  2244 birds were counted on 22/3 (ABC) making it the most numerous species of the day Date:  1998-03-22 00:00:00 Count:  2244
Updated count?
2244
Comments:  16 on an Albizia at Poyan, 10/2 (AC/LKS/AOY/SY) and 11 at Lorong Asrama, 10/2 (AC) Date:  1997-02-10 00:00:00 Count:  11
Updated count?
11
Comments:  ] hunting at Track 246, 18/1 (LKS/MS) Date:  1997-01-18 00:00:00 Count:  246
Updated count?
1
Comments:  A few at Track 246, 18/1 (GY Y/AL/LKS/NBC/MS/AT) Date:  1997-01-18 00:00:00 Count:  246
Updated count?
X
Comments:  Common at Sedili Track 246, 18/1 (GY Y/AL/LKS/NBC/MS/AT) Date:  1997-01-18 00:00:00 Count:  246
Updated count?
X
Comments:  ] heard at Track 246, 18/1 (LKS) Date:  1997-01-18 00:00:00 Count:  246
Updated count?
1
Comments:  Also seen on 10/2 (AC) Date:  2000-02-10 00:00:00 Count:  1999
Updated count?
1
Comments:  Also heard along Jenut Muda Trail, 22/2 (LKC) Date:  2000-02-22 00:00:00 Count:  11
Updated count?
X
1 at Panti Forest (Bunker Track), 16/3 (CA)
Ho

,A: Common Name,B: Genus,C: Species,D: Species Count,E: Species Comments,E1: Comments,F: Location Name,F1: Original Location (removed for upload),G: Latitude,H: Longitude,I: Observation Date,J: Start Time,K: State/Province,L: Country,M: Protocol,N: Number of Observers,O: Duration,P: All Observations Reported?,Q: Distance Covered,R: Area Covered,S: Checklist Comments
0,Banded Bay Cuckoo,,Cacomantis sonneratii,1,<b>Comments: </b>Singles seen at the summit of...,"Singles seen at the summit of Bukit Timah, 4/1...",Bukit Timah,Bukit Timah,,,1998-01-04,,,SG,historical,1,,N,,,These sightings are reproduced with permission...
1,Banded Bay Cuckoo,,Cacomantis sonneratii,1,<b>Comments: </b>Singles seen at the summit of...,"Singles seen at the summit of Bukit Timah, 4/1...",Sime Road,Sime Road,,,1998-01-11,,,SG,historical,,,N,,,These sightings are reproduced with permission...
2,Banded Bay Cuckoo,,Cacomantis sonneratii,1,<b>Comments: </b>Singles were also heard at Si...,Singles were also heard at Sime Road on 25/1 (...,Sime Road,Sime Road,,,1998-01-25,,,SG,historical,1,,N,,,These sightings are reproduced with permission...
3,Banded Bay Cuckoo,,Cacomantis sonneratii,1,<b>Comments: </b>Singles were also heard at Si...,Singles were also heard at Sime Road on 25/1 (...,Poyan (Tengeh) Reservoir (restricted access),Poyan,,,1998-01-29,,,SG,historical,2,,N,,,These sightings are reproduced with permission...
4,Banded Bay Cuckoo,,Cacomantis sonneratii,2,<b>Comments: </b>2 seen low in trees at Lorong...,"2 seen low in trees at Lorong Sesuai, near Buk...",Bukit Batok Nature Park,Lorong Sesuai,,,1998-02-15,,,SG,historical,,,N,,,These sightings are reproduced with permission...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2238,Yellow-vented Flowerpecker,,Dicaeum chrysorrheum,1,"<b>Comments: </b>1 feeding on a low bush, Pant...","1 feeding on a low bush, Panti Forest (Bunker ...",?,Panti Forest,,,2000-03-16,,,?,historical,1,,N,,,These sightings are reproduced with permission...
2239,Streaky-breasted Spiderhunter,,Arachnothera affinis,1,<b>Comments: </b>1 at Panti Forest (Bunker Tra...,"1 at Panti Forest (Bunker Track), 14/3 (LKC)",?,Panti Forest,,,2000-03-14,,,?,historical,1,,N,,,These sightings are reproduced with permission...
2240,Straw-headed Bulbul,,Pycnonotus zeylanicus,2,"<b>Comments: </b>At BBNP, 5 birds were counted...","At BBNP, 5 birds were counted on 5/2 (LKS/SA) ...",Bukit Batok Nature Park,BBNP,,,2000-02-05,,,SG,historical,2,,N,,,These sightings are reproduced with permission...
2241,Crested Fireback,,Lophura ignita,X,<b>Comments: </b>1 male foraging along Bukit T...,"1 male foraging along Bukit Teresik Trail, 21/...",?,Tahan River,,,2000-02-23,,,?,historical,1,,N,,,These sightings are reproduced with permission...


In [20]:
### COPY TO DRIVE

% cp "/content/df (10).csv" -r "/content/gdrive/My Drive/"
% cp "/content/output.csv" -r "/content/gdrive/My Drive/"
% cp "/content/cachedeng.npy" -r "/content/gdrive/My Drive/"
% cp "/content/cachedsci.npy" -r "/content/gdrive/My Drive/"
% cp "/content/output-spec.csv" -r "/content/gdrive/My Drive/"
% cp "/content/upload-spec.csv" -r "/content/gdrive/My Drive/"
% cp "/content/contributors.npy" -r "/content/gdrive/My Drive/"
% cp "/content/cachedpicks.npy" -r "/content/gdrive/My Drive/"
% cp "/content/lookup-hotspots.csv" -r "/content/gdrive/My Drive/"